<a href="https://colab.research.google.com/github/jaiklen/-Elliptical-Curve-Cryptography/blob/main/Hotel_Review_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy.random
import numpy as np
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [ ]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# reading the CSV file
csvFile = pd.read_csv('/content/drive/MyDrive/Q3Dataset.csv')
#perm=np.random.permutation(1000)
#csvFile=csvFile.iloc[perm]
In_data=csvFile.iloc[:,0]
out_label=csvFile.iloc[:,1]


# displaying the contents of the CSV file
print(csvFile.shape)
"""
X_train,X_test,y_train,y_test=train_test_split(In_data,out_label,test_size=0.2)

print(X_test.shape)
print(y_train.shape)
"""

(1000, 2)


'\nX_train,X_test,y_train,y_test=train_test_split(In_data,out_label,test_size=0.2)\n\nprint(X_test.shape)\nprint(y_train.shape)\n'

In [ ]:
tokenizer=RegexpTokenizer(r'\w+')
stop_words=set(stopwords.words('english'))
pp_Input=[]
for l in In_data:
  """changing to lower case"""
  l=l.lower() 

  """Removing Punctuation"""
  token=tokenizer.tokenize(l)

  """Removing Stop words"""
  pp_text=[word for word in token if word not in stop_words]

  Str=""
  for i in pp_text:
    Str+=i+" "

  pp_Input.append(Str)
  #print(Str)
#print(len(pp_Input))

In [ ]:
"""Word vectoriztion using TFID"""

vectorizer = TfidfVectorizer()
vectorizer.fit(pp_Input)
skl_output = vectorizer.transform(pp_Input)

In [ ]:
print(vectorizer.get_feature_names())
print(type(skl_output))
print(skl_output.shape)

['00', '10', '100', '11', '12', '15', '17', '1979', '20', '2007', '23', '30', '30s', '35', '40', '40min', '45', '4ths', '5lb', '70', '85', '90', '99', 'absolute', 'absolutely', 'absolutley', 'accident', 'accommodations', 'accomodate', 'accordingly', 'accountant', 'ache', 'acknowledged', 'across', 'actual', 'actually', 'added', 'affordable', 'afternoon', 'ago', 'ahead', 'airline', 'airport', 'ala', 'albondigas', 'allergy', 'almonds', 'almost', 'alone', 'also', 'although', 'always', 'amazing', 'ambiance', 'ambience', 'amount', 'ample', 'andddd', 'angry', 'annoying', 'another', 'anticipated', 'anymore', 'anyone', 'anything', 'anytime', 'anyway', 'anyways', 'apart', 'apologize', 'apology', 'app', 'appalling', 'apparently', 'appealing', 'appetite', 'appetizer', 'appetizers', 'apple', 'approval', 'area', 'arepas', 'aria', 'around', 'array', 'arrived', 'arrives', 'arriving', 'article', 'ask', 'asked', 'asking', 'assure', 'ate', 'atmosphere', 'atrocious', 'attached', 'attack', 'attention', 'at

In [ ]:
print(vectorizer.idf_)

[7.2156076  6.11699531 6.81014249 ... 7.2156076  6.29931687 6.29931687]


In [ ]:
"""Spliting into test and train test in 80:20 ratio"""
X_train,X_test=skl_output[0:800],skl_output[800:]
y_train,y_test=out_label[0:800],out_label[800:]

In [ ]:
uniqueValues, occurCount = np.unique(y_train, return_counts=True)
print(uniqueValues, occurCount)
uniqueValues, occurCount = np.unique(y_test, return_counts=True)
print(uniqueValues, occurCount)

[0 1] [348 452]
[0 1] [152  48]


In [ ]:
SupportVectorClassModel = SVC()
SupportVectorClassModel.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
#print(X_train.shape)

In [ ]:
y_pred = SupportVectorClassModel.predict(X_test)

In [ ]:
#print(y_pred.shape,y_test.shape)

In [ ]:
"""Function to calculate accuracy,F1 score and confusion matrix"""
def metrics(actual,predicted):
    actual=np.array(actual)
    predicted=np.array(predicted)
    total_data=len(actual)
    len_class= len(np.unique(actual))
    cm= np.zeros((len_class,len_class),int) #confusion matrix
    #print(cm.shape)
    for i in range(total_data):
      #print(actual[i],predicted[i])
      #print(cm[actual[i]][predicted[i]])
      cm[actual[i]][predicted[i]]=cm[actual[i]][predicted[i]]+1

    # F1-score for each class
    f1_score=np.zeros(len_class)

    for i in range(len_class):
        recall = cm[i][i] / np.sum(cm[i])
        precision = cm[i][i] / np.sum(cm, axis=0)[i]
        f1_score[i] = 2 * recall * precision / (recall + precision)

    # Accuracy
    Accuracy=np.trace(np.array(cm))/total_data
    print("F1-Score : \n", f1_score)
    print("Accuracy : \n", Accuracy)
    print("Confusion Matrix : \n", cm)
print("performance scores before Grid Serach")
metrics(y_test,y_pred)

performance scores before Grid Serach
F1-Score : 
 [0.68669528 0.56287425]
Accuracy : 
 0.635
Confusion Matrix : 
 [[80 72]
 [ 1 47]]


In [ ]:
"""performing Grid search CV here"""
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)




In [ ]:
#print(X_train[0].shape,X_train[1].shape)

In [ ]:
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.569, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.569, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.562, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.562, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.562, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.569, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.569, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.562, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.562, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.562, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.569, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   20.9s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)
print(grid.best_score_)

{'C': 1, 'gamma': 1, 'kernel': 'linear'}
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.76875


In [ ]:
"""Prediction using best hyper parameter"""
grid_predictions = grid.predict(X_test)
# print classification report
#print(classification_report(y_test, grid_predictions))
print("scores after Grid search CV")
metrics(y_test,grid_predictions)

scores after Grid search CV
F1-Score : 
 [0.82527881 0.64122137]
Accuracy : 
 0.765
Confusion Matrix : 
 [[111  41]
 [  6  42]]
